<a href="https://colab.research.google.com/github/Ashok54942/-FASTRACK-Ai-Driven-Smart-Attendance-System-/blob/main/FirstWeekTask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
import pandas as pd
import numpy as np

# Upload file
uploaded = files.upload()




Saving Cars Datasets 2025.csv to Cars Datasets 2025.csv


In [2]:
# Load dataset
df = pd.read_csv("Cars Datasets 2025.csv", encoding="latin1")

# View basic info
print(df.shape)
df.head()


(1218, 11)


,Company Names,Cars Names,Engines,CC/Battery Capacity,HorsePower,Total Speed,Performance(0 - 100 )KM/H,Cars Prices,Fuel Types,Seats,Torque
0,FERRARI,SF90 STRADALE,V8,3990 cc,963 hp,340 km/h,2.5 sec,"$1,100,000",plug in hyrbrid,2,800 Nm
1,ROLLS ROYCE,PHANTOM,V12,6749 cc,563 hp,250 km/h,5.3 sec,"$460,000",Petrol,5,900 Nm
2,Ford,KA+,1.2L Petrol,"1,200 cc",70-85 hp,165 km/h,10.5 sec,"$12,000-$15,000",Petrol,5,100 - 140 Nm
3,MERCEDES,GT 63 S,V8,"3,982 cc",630 hp,250 km/h,3.2 sec,"$161,000",Petrol,4,900 Nm
4,AUDI,AUDI R8 Gt,V10,"5,204 cc",602 hp,320 km/h,3.6 sec,"$253,290",Petrol,2,560 Nm


In [3]:
# Column names
print(df.columns.tolist())

# Summary
df.info()

# Missing values
df.isnull().sum()

# Quick statistics
df.describe(include="all")


['Company Names', 'Cars Names', 'Engines', 'CC/Battery Capacity', 'HorsePower', 'Total Speed', 'Performance(0 - 100 )KM/H', 'Cars Prices', 'Fuel Types', 'Seats', 'Torque']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1218 entries, 0 to 1217
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Company Names              1218 non-null   object
 1   Cars Names                 1218 non-null   object
 2   Engines                    1218 non-null   object
 3   CC/Battery Capacity        1215 non-null   object
 4   HorsePower                 1218 non-null   object
 5   Total Speed                1218 non-null   object
 6   Performance(0 - 100 )KM/H  1212 non-null   object
 7   Cars Prices                1218 non-null   object
 8   Fuel Types                 1218 non-null   object
 9   Seats                      1218 non-null   object
 10  Torque                     1217 non-null   object
dtypes

,Company Names,Cars Names,Engines,CC/Battery Capacity,HorsePower,Total Speed,Performance(0 - 100 )KM/H,Cars Prices,Fuel Types,Seats,Torque
count,1218,1218,1218,1215,1218,1218,1212,1218,1218,1218,1217
unique,37,1201,356,311,456,114,180,535,23,19,263
top,Nissan,Mistral,I4,1984 cc,355 hp,250 km/h,6.5 sec,"$35,000",Petrol,5,400 Nm
freq,149,2,64,31,23,145,45,36,871,692,72


In [4]:
# Standardize column names
df.columns = (
    df.columns
    .str.strip()
    .str.replace(" ", "_")
    .str.replace("/", "_")
    .str.replace("-", "_")
)
print(df.columns.tolist())


['Company_Names', 'Cars_Names', 'Engines', 'CC_Battery_Capacity', 'HorsePower', 'Total_Speed', 'Performance(0___100_)KM_H', 'Cars_Prices', 'Fuel_Types', 'Seats', 'Torque']


In [6]:
# HorsePower cleaning
df["HorsePower_Clean"] = (
    df["HorsePower"]
    .astype(str)
    .str.extractall(r'(\d+)')[0]   # get the first capture group
    .astype(float)                 # convert numbers to float
    .groupby(level=0)              # group by original row index
    .mean()                        # take mean if multiple numbers
)



In [8]:
# Check exact column names
print(df.columns)

# Use the correct column
df["Engine_CC_Clean"] = (
    df["CC_Battery_Capacity"]      # ✅ correct column name
    .astype(str)
    .str.extract(r"(\d+)")         # extract digits
    .astype(float)
)



Index(['Company_Names', 'Cars_Names', 'Engines', 'CC_Battery_Capacity',
       'HorsePower', 'Total_Speed', 'Performance(0___100_)KM_H', 'Cars_Prices',
       'Fuel_Types', 'Seats', 'Torque', 'HorsePower_Clean'],
      dtype='object')


In [10]:
# Torque cleaning
df["Torque_Clean"] = (
    df["Torque"]
    .astype(str)
    .str.extractall(r"(\d+)")[0]   # extract the first capture group
    .astype(float)                 # convert BEFORE groupby
    .groupby(level=0)
    .mean()                        # take mean if multiple numbers exist
)



In [11]:
def clean_price(x):
    x = str(x).replace(",", "").replace("$", "")
    if "-" in x:
        try:
            nums = [int(i) for i in x.split("-") if i.isdigit()]
            return np.mean(nums)
        except:
            return np.nan
    return pd.to_numeric(x, errors="coerce")

df["Car_Price_Clean"] = df["Cars_Prices"].apply(clean_price)


/usr/local/lib/python3.12/dist-packages/numpy/_core/fromnumeric.py:3596: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [12]:
for col in ["HorsePower_Clean", "Engine_CC_Clean", "Torque_Clean", "Car_Price_Clean"]:
    df[col].fillna(df[col].median(), inplace=True)


/tmp/ipython-input-3765426689.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].median(), inplace=True)


In [13]:
def remove_outliers(col):
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower, upper = Q1 - 1.5*IQR, Q3 + 1.5*IQR
    return df[(df[col] >= lower) & (df[col] <= upper)]

df = remove_outliers("Car_Price_Clean")


In [14]:
df.head()
df.to_csv("Cleaned_Cars_Dataset.csv", index=False)
